In [13]:
import pandas as pd
import requests
import xmltodict

In [14]:
def get_values_of_key(dictionary: dict, key: str):
    values = []
    for k, v in dictionary.items():
        if k == key:
            values.append(v)
        elif isinstance(v, dict):
            values.extend(get_values_of_key(v, key))
        elif isinstance(v, list):
            for item in v:
                if isinstance(item, dict):
                    values.extend(get_values_of_key(item, key))
    return values

In [15]:
language = "Marathi"
suffix = ""
response = requests.get(
    f"https://raw.githubusercontent.com/christos-c/bible-corpus/master/bibles/{language}{suffix}.xml"
)
data_xml = response.text

In [16]:
data_dict = xmltodict.parse(data_xml)
text_dict = data_dict.get("cesDoc", {}).get("text", {})

In [17]:
text: list[str] = get_values_of_key(text_dict, "#text")
sentences: list = list(
    map(lambda sentence: {"Text": sentence, "Language": language}, text)
)

In [18]:
df = pd.DataFrame(sentences)
df = df[df["Text"] != ""]
df.to_csv(f"./training/{language}.csv", index=False)